train_model / GMM_HMM.py

In [ ]:
from hmmlearn import hmm


def GMM_HMM(O, lengths, n_states, v_type, n_iter, verbose=True):

    # model = hmm.GMMHMM(n_components=n_states, covariance_type=v_type, n_mix=4, n_iter=n_iter, verbose=verbose).fit(O, lengths)
    model = hmm.GaussianHMM(n_components=n_states, covariance_type=v_type, n_iter=n_iter, verbose=verbose).fit(O, lengths)

    return model


train_model / LSTM.py

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.advanced_activations import LeakyReLU
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelBinarizer
from public_tool.bagging_balance_weight import bagging_balance_weight
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from public_tool.form_index import form_index
from public_tool.random_cut import random_cut
from public_tool.form_accuracy import form_accuracy
from keras.models import load_model
import pickle


def form_LSTM_dataset(X, y, lengths, T):
    result_X = []
    result_y = []
    for i in range(len(lengths)):
        begin_index, end_index = form_index(lengths, i)
        now_X = X[begin_index:end_index]
        now_y = y[begin_index:end_index]

        for j in range(len(now_y) - T):
            result_X.append(now_X[j:j + T])
            result_y.append(now_y[j + T])

    result_X = np.array(result_X)

    return result_X, result_y


def self_LSTM(X, y, lengths, file_name):
    # normalization
    mms = MinMaxScaler()
    X = mms.fit_transform(X)

    T = 10
    X_LSTM, y_LSTM = form_LSTM_dataset(X, y, lengths, T)

    # one hot
    lb = LabelBinarizer()
    y_LSTM = lb.fit_transform(y_LSTM)

    # random cut
    X_train, y_train, X_valid, y_valid = random_cut(X_LSTM, y_LSTM, 5)
    X_train, y_train = bagging_balance_weight(X_train, y_train)

    # creat and fit model
    model = Sequential()

    model.add(Dropout(0.2))
    model.add(LSTM(40))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.02))

    model.add(Dropout(0.2))
    model.add(Dense(30, kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.02))

    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))

    sgd = SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # es = EarlyStopping(monitor='val_loss', min_delta=3e-4, patience=10, verbose=1, mode='auto')
    model.fit(X_train,
              y_train,
              batch_size=int(X_train.shape[0] / 5),
              epochs=10000,
              verbose=2,
              validation_split=0.0,
              validation_data=(X_valid, y_valid),
              shuffle=True,
              class_weight=None,
              sample_weight=None,
              initial_epoch=0,
              callbacks=[])

    print('train    ', form_accuracy(X_train, y_train, model))
    print('valid   ', form_accuracy(X_valid, y_valid, model))

    mms_file_name = 'C:/Users/Administrator/Desktop/HMM_program/save' + file_name + '_mms.pkl'
    pickle.dump(mms, open(mms_file_name, 'wb'))
    model_file_name = 'C:/Users/Administrator/Desktop/HMM_program/save' + file_name + '_model.h5'
    model.save(model_file_name)


train_model / train_HMM_model.py

In [ ]:
from dataset_code.process_on_raw_data import form_raw_dataset
from public_tool.form_model_dataset import form_model_dataset
from public_tool.solve_on_outlier import solve_on_outlier
from train_model.GMM_HMM import GMM_HMM
from train_model.XGB_HMM import XGB_HMM
import pickle
from dataset_code import HMM_duoyinzi, HMM_hangqing
import os


def train_HMM_model(n_states):
    # train the hnagqing or the duoyinzi GMM_HMM model and XGB_HMM model

    # 1 hangqing
    # 1.1 generate the hangqing dataset
    if not (os.path.exists('C:/Users/Administrator/Desktop/HMM_program/save/hangqing_GMM_HMM_model.pkl') and os.path.exists('C:/Users/Administrator/Desktop/HMM_program/save/hangqing_XGB_HMM_model.pkl')):
        feature_col = ['preClosePrice', 'openPrice', 'closePrice', 'turnoverVol', 'highestPrice', 'lowestPrice']
        dataset, label, lengths, col_nan_record = form_raw_dataset(feature_col, label_length=5)
        solved_dataset, allow_flag = HMM_hangqing.solve_on_raw_data(dataset, lengths, feature_col)
        X_train, y_train, lengths_train = form_model_dataset(solved_dataset, label, allow_flag, lengths)
        X_train = solve_on_outlier(X_train, lengths_train)

        # 1.2 train and save the GMM_HMM model
        print('training hangqing GMM_HMM model...')
        temp = GMM_HMM(X_train, lengths_train, n_states, 'diag', 1000, True)
        pickle.dump(temp, open('C:/Users/Administrator/Desktop/HMM_program/save/hangqing_GMM_HMM_model.pkl', 'wb'))

        # 1.3 train and save the XGB_HMM model
        print('training hangqing XGB_HMM model...')
        A, xgb_model, pi = XGB_HMM(X_train, lengths_train)
        pickle.dump([A, xgb_model, pi], open('C:/Users/Administrator/Desktop/HMM_program/save/hangqing_XGB_HMM_model.pkl', 'wb'))

    # 2 duoyinzi
    print('training duoyinzi...')
    if not (os.path.exists('C:/Users/Administrator/Desktop/HMM_program/save/duoyinzi_GMM_HMM_model.pkl') and os.path.exists('C:/Users/Administrator/Desktop/HMM_program/save/duoyinzi_XGB_HMM_model.pkl')):
        score, feature_name = HMM_duoyinzi.load_duoyinzi_single_score()
        feature_col_duoyinzi = HMM_duoyinzi.type_filter(score, feature_name, 0.1)  # there are 7 kinds of duoyinzi
        GMM_model_list = []
        XGB_model_list = []
        for i in range(len(feature_col_duoyinzi)):
            feature_col = feature_col_duoyinzi[i]
            # 2.1 generate the duoyinzi dataset
            dataset, label, lengths, col_nan_record = form_raw_dataset(feature_col, label_length=5)
            print(sum(lengths))
            print(dataset.shape[0])
            solved_dataset, allow_flag = HMM_duoyinzi.solve_on_raw_data(dataset, lengths, feature_col, feature_col)
            X_train, label_train, lengths_train = form_model_dataset(solved_dataset, label, allow_flag, lengths)
            pickle.dump([X_train, lengths_train], open('C:/Users/Administrator/Desktop/HMM_program/save/temp.pkl', 'wb'))
            X_train = solve_on_outlier(X_train, lengths_train)

            # 2.2 train and record the GMM_HMM model
            print('training duoyinzi GMM_HMM model %s...' % (i+1))
            pickle.dump([X_train, lengths_train], open('C:/Users/Administrator/Desktop/HMM_program/save/temp1.pkl', 'wb'))
            print(X_train.shape[0])
            print(sum(lengths_train))
            temp = GMM_HMM(X_train, lengths_train, n_states, 'diag', 1000, True)
            GMM_model_list.append(temp)

            # 2.3 train and record the XGB_HMM model
            print('training duoyinzi XGB_HMM model %s...' % (i+1))
            A, xgb_model, pi = XGB_HMM(X_train, lengths_train)
            XGB_model_list.append([A, xgb_model, pi])

        # 2.4 save the model
        pickle.dump(GMM_model_list, open('C:/Users/Administrator/Desktop/HMM_program/save/duoyinzi_GMM_HMM_model.pkl', 'wb'))
        pickle.dump(XGB_model_list, open('C:/Users/Administrator/Desktop/HMM_program/save/duoyinzi_XGB_HMM_model.pkl', 'wb'))


train_model / train_LSTM_model.py

In [ ]:
from dataset_code.process_on_raw_data import form_raw_dataset
import pickle
from dataset_code import HMM_duoyinzi, HMM_hangqing
from dataset_code.pred_proba_GMM import pred_proba_GMM
from dataset_code.combine import combine
from dataset_code.pred_proba_XGB import pred_proba_XGB
from train_model.LSTM import self_LSTM


def train_LSTM_model():
    # train the LSTM model based on state_proba formed by GMM_HMM or XGB_HMM

    # generate the dataset

    feature_col_hangqing = ['preClosePrice', 'openPrice', 'closePrice', 'turnoverVol', 'highestPrice', 'lowestPrice']
    score, feature_name = HMM_duoyinzi.load_duoyinzi_single_score()
    feature_col_duoyinzi = HMM_duoyinzi.type_filter(score, feature_name, 0.1)
    feature_col = feature_col_hangqing
    _ = [[feature_col.append(j) for j in i] for i in feature_col_duoyinzi]
    dataset, label, lengths, col_nan_record = form_raw_dataset(feature_col, 5)

    # 1 by GMM_HMM
    # 1.1 hangqing
    solved_dataset1, allow_flag1 = HMM_hangqing.solve_on_raw_data(dataset, lengths, feature_col)
    model = pickle.load(open('C:/Users/Administrator/Desktop/HMM_program/save/hangqing_GMM_HMM_model.pkl', 'rb'))
    pred_proba1 = pred_proba_GMM(model, solved_dataset1, allow_flag1, lengths)

    # 1.2 duoyinzi
    pred_proba2 = []
    allow_flag2 = []
    model = pickle.load(open('C:/Users/Administrator/Desktop/HMM_program/save/duoyinzi_GMM_HMM_model.pkl', 'rb'))
    for i in range(len(feature_col_duoyinzi)):
        temp_solved_dataset, temp_allow_flag = HMM_duoyinzi.solve_on_raw_data(dataset, lengths, feature_col, feature_col_duoyinzi[i])
        temp_model = model[i]
        temp_pred_proba = pred_proba_GMM(temp_model, temp_solved_dataset, temp_allow_flag, lengths)
        pred_proba2.append(temp_pred_proba)
        allow_flag2.append(temp_allow_flag)

    # 1.3 combine two type state_proba
    final_X, final_y, final_lengths = combine(pred_proba1, pred_proba2, allow_flag1, allow_flag2, label, lengths)

    # 1.4 train LSTM model
    self_LSTM(final_X, final_y, final_lengths, 'GMM_HMM_LSTM')

    # 2 by XGB_HMM
    # 2.1 hangqing
    solved_dataset1, allow_flag1 = HMM_hangqing.solve_on_raw_data(dataset, lengths, feature_col)
    temp = pickle.load(open('C:/Users/Administrator/Desktop/HMM_program/save/hangqing_XGB_HMM_model.pkl', 'rb'))
    A, model, pi = temp[0], temp[1], temp[2]
    pred_proba1 = pred_proba_XGB(A, model, pi, solved_dataset1, allow_flag1, lengths)

    # 2.2 duoyinzi
    pred_proba2 = []
    allow_flag2 = []
    model = pickle.load(open('C:/Users/Administrator/Desktop/HMM_program/save/duoyinzi_XGB_HMM_model.pkl', 'rb'))
    for i in range(len(feature_col_duoyinzi)):
        temp_solved_dataset, temp_allow_flag = HMM_duoyinzi.solve_on_raw_data(dataset, lengths, feature_col, feature_col_duoyinzi[i])
        temp_A, temp_model, temp_pi = model[i][0], model[i][1], model[i][2]
        temp_pred_proba = pred_proba_XGB(temp_A, temp_model, temp_pi, temp_solved_dataset, temp_allow_flag, lengths)
        pred_proba2.append(temp_pred_proba)
        allow_flag2.append(temp_allow_flag)

    # 2.3 combine two type state_proba
    final_X, final_y, final_lengths = combine(pred_proba1, pred_proba2, allow_flag1, allow_flag2, label, lengths)

    # 2.4 train LSTM model
    self_LSTM(final_X, final_y, final_lengths, 'XGB_HMM_LSTM')


train_model / XGB_HMM.py

In [ ]:
import numpy as np
from XGB_HMM.GMM_HMM import GMM_HMM
from XGB_HMM.re_estimate import re_estimate
from XGB_HMM.predict import self_pred
from XGB_HMM.xgb import self_xgb


def XGB_HMM(O, lengths, verbose=True):

    n_states = 3
    stop_flag = 0
    iteration = 1
    log_likelihood = -np.inf
    min_delta = 1e-4

    S, A, gamma = GMM_HMM(O, lengths, n_states, verbose=True)
    prior_pi = np.array([sum(S == i) / len(S) for i in range(n_states)])
    # model = self_xgb(O, gamma, n_states)
    model = 1
    # B_Matrix = form_B_matrix_by_DNN(model, O, prior_pi)
    B_Matrix = gamma / prior_pi

    record_log_likelihood = []
    best_result = []  # record the result A, B(xgb model), prior pi, best_log_likelihood

    while stop_flag <= 3:

        A, gamma = re_estimate(A, B_Matrix, prior_pi, lengths)
        # pickle.dump([O, gamma], open('C:/Users/Administrator/Desktop/HMM_program/save/temp.pkl', 'wb'))
        # model = self_xgb(O, gamma, n_states)

        # model = self_DNN(O, gamma)
        # B_Matrix = form_B_matrix_by_DNN(model, O, prior_pi)

        B_Matrix = gamma / prior_pi

        new_S, _, new_log_likelihood = self_pred(B_Matrix, lengths, A, prior_pi)

        record_log_likelihood.append(new_log_likelihood)

        if len(best_result) == 0:
            best_result = [A, model, prior_pi, new_log_likelihood]
        elif new_log_likelihood > best_result[3]:
            best_result = [A, model, prior_pi, new_log_likelihood]
            temp = gamma

        if new_log_likelihood - log_likelihood <= min_delta:
            stop_flag += 1
        else:
            stop_flag = 0

        log_likelihood = new_log_likelihood
        iteration += 1

        if verbose:
            print(new_log_likelihood)

    model = self_xgb(O, temp, n_states)
    best_result[1] = model

    return best_result[0], best_result[1], best_result[2]


XGB_HMM / from_B_matrix_by_XGB.py

In [ ]:
import xgboost as xgb


def form_B_matrix_by_XGB(model, O, pi):
    pred = model.predict(xgb.DMatrix(O))

    B = pred/pi

    return B
